In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
train_data = pd.read_table('ratings_train.txt')

In [22]:
# document 열의 중복 제거 (원본 값 변경 허용)
train_data.drop_duplicates(subset=['document'], inplace=True)

# document 열의 NaN 제거
train_data = train_data.dropna(how = 'any')

# document 열의 값들 한국어와 공백만 남기고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-힣 ]", "")

# 빈칸 document값은 NaN으로 변환 (원본 값 변경 허용)
train_data['document'].replace("", np.nan, inplace=True)

# document 열의 NaN 제거
train_data = train_data.dropna(how = 'any')

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_9376\917818701.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-힣 ]", "")


In [23]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()

X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    X_train.append(stopwords_removed_sentence)

100%|██████████| 145804/145804 [06:07<00:00, 396.29it/s]


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

tokenizer.word_counts.items()

In [25]:
tokenizer = Tokenizer()
print(len(X_train))
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 1

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
y_train = np.array(train_data['label'])

drop_train = [index for (index, sentence) in enumerate(X_train) if len(sentence) == 0] # 빈 값 제거
X_train = np.delete(X_train, drop_train, axis=0)

145804


c:\Python310\lib\site-packages\numpy\lib\function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [16]:
max_len = 0

for a in X_train:
  if max_len < len(a): max_len = len(a)

drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) == 0]
print(len(X_train), len(y_train))
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

X_train = pad_sequences(X_train, maxlen=max_len)

145164 145804


In [26]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [19]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_train, y_train)[1]))

ValueError: Data cardinality is ambiguous:
  x sizes: 145164
  y sizes: 145804
Make sure all arrays contain the same number of samples.